In [ ]:
import numpy as  np
from tensorflow import keras 
import glob
from keras.callbacks import ModelCheckpoint
!pip install rasterio
!pip install segmentation_models
import segmentation_models as sm
from keras.callbacks import ModelCheckpoint
import rasterio
from segmentation_models.losses  import bce_jaccard_loss
from segmentation_models.metrics import iou_score
from keras.optimizers import Adam


In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
gauth = GoogleAuth()           
drive = GoogleDrive(gauth) 
import os
from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
root_path = '/content/gdrive/My Drive/handin3/datasetfinal'  #change dir to your project folder
os.chdir(root_path)
os.getcwd() 


In [ ]:
mask_list = (f for f in sorted(glob.glob(root_path + "/ma*")))
counts_list = (f for f in sorted(glob.glob(root_path + "/co*")))
patch_list = (f for f in sorted(glob.glob(root_path + "/pa*")))

In [ ]:
X = np.zeros((97, 256, 256, 4))
y = np.zeros((97, 256, 256))
for count, image in enumerate(patch_list):
    with rasterio.open(image, 'r') as src:
        #counts
        data = src.read()
        data = np.swapaxes(data,0,1)
        data = np.swapaxes(data,1,2)
        #swap axes so we could get (x,256,256)
        X[count, :, :, 0:3] = data[:, :, 0:3]
    

        #X[ number of photo,256,256,bands ]
for count,image in enumerate(counts_list):
    with rasterio.open(image, 'r') as src:
        data = src.read()
        data = data[0,:, :]
        unique,counts=np.unique(data,return_counts=True)
        #get the unique values and the encounters in each image
        if counts[0] > counts[1]:
            #if the 0 are more than the other 
                data[data[:,:] == unique[0]] = 0
                data[data[:,:] == unique[1]]  = 1
        else:
                data[data[:,:] == unique[0]] = 1
                data[data[:,:] == unique[1]]  = 0
        if data.shape[0] == 250:
            #for corrupt images fill the matrix with 0
              data = np.full((256,256), 0)
        X[count, :, :, 3] = data
        #the data we want are stored at the last band of X matrix 
for count, image in enumerate(mask_list):
    with rasterio.open(image, 'r')as src:
        #for the masks
        data  = src.read()
        data = data[0, :, :]
        data = data.astype(np.float32)
        unique,counts=np.unique(data,return_counts=True)
        #sort the columns with most elements, keep 2 
        args = counts.argsort()[-2:][::-1]
        data[data[:, :] != unique[args[1]]] = np.nan
        data[data[:, :] == unique[args[1]]] = 1
        data[np.isnan(data[:, :])] = 0
        y[count, :, :] = data
        #y is a matrix that contains the labels for each mask

In [ ]:
BACKBONE = 'resnet34'
filepath='loss2.hdf5'
np.random.seed(2)
# lists to keep the tests splits 
foldsXX = []
foldsYY = []
iou_scores = []
X = X[:, :, :, 0:3]
for seed in np.random.randint(1000, size=2):
    np.random.seed(seed)
    idx = np.random.permutation(len(X))
    # scramble the indeces of the X matrix 
    X,y = X[idx], y[idx]
    foldsX = []
    foldsY = []
    for fold in range(5):
        start = int((len(X)/5)) * fold
        end = int((len(X)/5)) * (fold+1)
        #5fold split here 
        foldsX.append(X[start:end, :, :, :])
        foldsY.append(y[start:end, :, :])
    foldsXX.append(foldsX)
    foldsYY.append(foldsY)
    # we keep the sets of trials 
    for fold in range(5):
        X_val = foldsX[fold]
        y_val = foldsY[fold]
        X_train = np.zeros((76, 256, 256, 3))
        y_train = np.zeros((76, 256, 256))
        counter = 0
        for fold_train in range(5):
            if fold_train != fold:
                X_train[(19*counter):(19*(counter+1)), :, :, :] = foldsX[fold_train]
                y_train[(19*counter):(19*(counter+1)), :, :] = foldsY[fold_train]
    
    y_train = y_train.reshape((76, 256, 256, 1))
    y_val = y_val.reshape((19, 256, 256, 1))
    model = sm.Unet(BACKBONE, encoder_weights= None, input_shape= (None, None, 3))
    adam = Adam(learning_rate=0.0001)
    model.compile(
        'Adam',
        loss=bce_jaccard_loss,
        metrics=[iou_score],
        )
    checkpoint = ModelCheckpoint(filepath, monitor='val_iou_score', verbose=1, save_best_only=True, mode='max')
    model.fit(X_train,y_train, batch_size=32, epochs=100, validation_data=(X_val, y_val), callbacks=[checkpoint])
    model.load_weights(filepath)
    print(model.evaluate(X_val, y_val)[1])
    iou_scores.append(model.evaluate(X_val, y_val)[1])


In [ ]:
print(model.evaluate(X_val, y_val)[1])
iou_scores.append(model.evaluate(X_val, y_val)[1])
print(iou_scores)
print(model.evaluate(X_val, y_val)[1])
iou_scores.append(model.evaluate(X_val, y_val)[1])
print(iou_scores)
ion = np.asarray(iou_scores)
print(ion)
print(ion.shape)
